In [1]:
import sys, os

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
fake_path = os.path.join(os.getcwd(),r'..\Fake.br-Corpus-master\full_texts\fake')
true_path = os.path.join(os.getcwd(),r'..\Fake.br-Corpus-master\full_texts\true')

In [22]:
df_corpus = pd.DataFrame(columns=['sample','label'])

for filename in tqdm(os.listdir(fake_path)):
    f = open(os.path.join(fake_path,filename), encoding="utf8")
    sample = f.read()

    df = pd.DataFrame([[sample,1]],columns=['sample','label'])

    df_corpus = df_corpus.append(df, ignore_index=True)

for filename in tqdm(os.listdir(true_path)):
    f = open(os.path.join(true_path,filename), encoding="utf8")
    sample = f.read()

    df = pd.DataFrame([[sample,0]],columns=['sample','label'])

    df_corpus = df_corpus.append(df, ignore_index=True)

df_corpus.to_pickle('./Dados/raw_corpus.pkl')


100%|██████████| 3600/3600 [01:28<00:00, 40.88it/s]
